In [1]:
import PIL
import numpy as np
import torch
import torchvision
from torchvision.transforms.v2 import Compose, GaussianBlur, RandomEqualize, RandomSolarize, RandomApply
import wandb
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from tasks.SRDiffTrainer import SRDiffTrainer
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

#Data
lr_size = 64
hr_size = 256
batch_size = 20
dataset_dir = 'C:\\Users\\adrianperera\\Desktop\\dataset_tfg'

transforms = Compose(
    [RandomApply(transforms= [GaussianBlur(7)], p = 0.5),
    RandomEqualize()]
)

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = None, aux_sat_prob= 0.4, sat_dataset_path= "C:\\Users\\adrianperera\\Desktop\\dataset_tfg\\satelite_dataset\\64_256")
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device(0)

In [2]:
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)

GaussianDiffusion(
  (denoise_fn): Unet(
    (cond_proj): ConvTranspose2d(96, 64, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
    (time_pos_emb): SinusoidalPosEmb()
    (mlp): Sequential(
      (0): Linear(in_features=64, out_features=256, bias=True)
      (1): Mish()
      (2): Linear(in_features=256, out_features=64, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0): ResnetBlock(
          (mlp): Sequential(
            (0): Mish()
            (1): Linear(in_features=64, out_features=64, bias=True)
          )
          (block1): Block(
            (block): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
              (2): Mish()
            )
          )
          (block2): Block(
            (block): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
              (2): Mish()
            )
 

In [3]:
model_name = "SRDIFF standart version 1.5"
hyperparams = {
    "lr":0.00002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 1000000,
    "mode": "min",
    "factor":0.5,
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 1,
    "use_rrdb":True,
    "fix_rrdb":True,
    "aux_l1_loss":False,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":False
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
params = list(model.named_parameters())
if not hyperparams['fix_rrdb']:
    params = [p for p in params if 'rrdb' not in p[0]]
params = [p[1] for p in params]
optimizer =  torch.optim.Adam(params, lr=hyperparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["factor"])
torch.backends.cudnn.benchmark = True
project_name = "SRDiff experiments"
run_name = model_name
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=model_name, device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"],
                        grad_acum=hyperparams["grad_acum"])

trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader, step)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff\\depured tests\\")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

wandb: Currently logged in as: adrianpereramoreno (ladoscuro). Use `wandb login --relogin` to force relogin


100%|██████████| 40/40 [08:38<00:00, 12.97s/batch, psnr=17.9, ssim=0.536]


psnr,▁
ssim,▁
train_loss,█▆▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,15.34541
ssim,0.5776
train_loss,0.09972
validation_loss,0.10424


In [4]:
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)

model_name = "SRDIFF standart version 2.5"
hyperparams = {
    "lr":0.00002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 1000000,
    "mode": "min",
    "factor":0.5,
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 1,
    "use_rrdb":True,
    "fix_rrdb":True,
    "aux_l1_loss":True,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":False
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
params = list(model.named_parameters())
if not hyperparams['fix_rrdb']:
    params = [p for p in params if 'rrdb' not in p[0]]
params = [p[1] for p in params]
optimizer =  torch.optim.Adam(params, lr=hyperparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["factor"])
torch.backends.cudnn.benchmark = True
project_name = "SRDiff experiments"
run_name = model_name
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=model_name, device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"],
                        grad_acum=hyperparams["grad_acum"])

trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader, step)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff\\depured tests\\")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

100%|██████████| 40/40 [08:40<00:00, 13.02s/batch, psnr=17.8, ssim=0.663]


psnr,▁
ssim,▁
train_loss,█▅▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▆▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,18.35708
ssim,0.64004
train_loss,0.09801
validation_loss,0.08828


In [5]:
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)

model_name = "SRDIFF standart version 3.5"
hyperparams = {
    "lr":0.00002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 1000000,
    "mode": "min",
    "factor":0.5,
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 1,
    "use_rrdb":True,
    "fix_rrdb":True,
    "aux_l1_loss":True,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":True
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
params = list(model.named_parameters())
if not hyperparams['fix_rrdb']:
    params = [p for p in params if 'rrdb' not in p[0]]
params = [p[1] for p in params]
optimizer =  torch.optim.Adam(params, lr=hyperparams['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["factor"])
torch.backends.cudnn.benchmark = True
project_name = "SRDiff experiments"
run_name = model_name
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name=model_name, device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"],
                        grad_acum=hyperparams["grad_acum"])

trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader, step)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff\\depured tests\\")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

100%|██████████| 40/40 [08:36<00:00, 12.92s/batch, psnr=17.7, ssim=0.67] 


psnr,▁
ssim,▁
train_loss,█▆▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▆▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,16.3327
ssim,0.61453
train_loss,0.09854
validation_loss,0.09737


In [6]:
from models.SR3Builder import SR3Builder
from tasks.SR3Trainer import SR3Trainer


lr_size = 64
hr_size = 256
batch_size = 128
dataset_dir = 'C:\\Users\\adrianperera\\Desktop\\dataset_tfg'

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = None, aux_sat_prob= 0.5, sat_dataset_path= "C:\\Users\\adrianperera\\Desktop\\dataset_tfg\\satelite_dataset\\64_256")
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device(0)

model_builder = SR3Builder()
model_builder = model_builder.set_standart()
model_builder = model_builder.set_sample_steps(1000)
model_builder = model_builder.set_losstype("l1")
model = model_builder.build()
model.to(device)

hyperparams = {
    "lr":0.0002,
    "epochs":100,
    "eta_min":1e-7,
    "decay_steps": 100000,
    "gamma" : 0.5,  
    "model" : "SR3",
    "grad_acum": 0,
    "ddp": False,
    "batch_size":batch_size
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["gamma"])

project_name = "SR model benchmarking"
run_name = "SR3 standart l1, 128Batch, 100k sampling"
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)

trainer = SR3Trainer(metrics_used=("ssim", "psnr"), model_name="SR3 Standart l1 DA 128b 100k sampling", grad_acum=hyperparams["grad_acum"])
trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for epoch in range(hyperparams["epochs"]):  
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    
    train_loss = trainer.train(train_dataloader, epoch)
    torch.cuda.empty_cache()
    if epoch % 10 == 0:
        trainer.save_model("saved models\\SR3")

    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()   

100%|██████████| 7/7 [12:14<00:00, 104.95s/batch, psnr=3.09, ssim=0.00183]


psnr,▁
ssim,▁
train_loss,██▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
validation_loss,██▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
psnr,3.81056
ssim,0.00199
train_loss,0.26088
validation_loss,0.26101
